In [ ]:
# 2/1-2/29
#一个部门一个表
# D:\0\zhs\数据统计\打卡报表\打卡报表.xlsx

In [1]:
import pandas as pd
import logging
import sys
import calendar
import xlsxwriter
from datetime import datetime

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 5000)
pd.set_option('display.max_colwidth', 1000)


In [2]:
# 读取打卡表格xlsx
xlx_path = r'D:\0\zhs\数据统计\打卡报表\广告分公司2020年3月考勤日报.xlsx'
sheet_name = '2020年3月考勤'
report_path = r'D:\0\zhs\数据统计\打卡报表\广告分公司3月打卡报表.xlsx'
with pd.ExcelFile(xlx_path) as xls:
        df = pd.read_excel(xls, sheet_name)
print(df.head(10))
print(df.index)
# 获取部门总数及人数总数

# 获取报表月份及最大天数
time_str = df[:1].values[0][0]
# print(time_str)
# 报表年份
year = int(time_str[:4])
# print(year)
# 报表月份
month = int(time_str[5:7])
# print(month)
# 最大天数
day_max = calendar.monthrange(year,month)[1]
# print(day_max)
# 报表日期
tb_date = time_str[:7] + '/01 -- ' + time_str[:7] + '/' + str(day_max)
tb_date = tb_date.replace('/', '-')
print(tb_date)

           时间      部门   姓名 上班打卡时间 下班打卡时间
0  2020/03/02  户外新媒体部  蔡朝博  08:29  17:54
1  2020/03/03  户外新媒体部  蔡朝博  08:26  17:57
2  2020/03/04  户外新媒体部  蔡朝博  07:54  18:16
3  2020/03/05  户外新媒体部  蔡朝博  07:52  17:41
4  2020/03/06  户外新媒体部  蔡朝博  08:27  17:30
5  2020/03/09  户外新媒体部  蔡朝博  08:36  18:20
6  2020/03/10  户外新媒体部  蔡朝博  08:21  18:03
7  2020/03/11  户外新媒体部  蔡朝博  08:29  17:55
8  2020/03/12  户外新媒体部  蔡朝博  08:19     --
9  2020/03/13  户外新媒体部  蔡朝博  08:26  18:27
RangeIndex(start=0, stop=2287, step=1)
2020-03-01 -- 2020-03-31


In [4]:
# 列出所有部门信息
# print(df.groupby('部门').size())
# 抽取部门名和姓名索引

alldn_df = pd.DataFrame(df.groupby(['部门', '姓名']).size())

dn_index = alldn_df.index
# 统计共多少人
# print(len(dn_index))

workbook = xlsxwriter.Workbook(report_path)

def get_day_and_week(day):
    # 获取日01、02
    if day>9:
        cell_day = str(day)
    else:
        cell_day = '0' + str(day)
    # 获取周一、六
    week_day_dict = {
        0 : '一',
        1 : '二',
        2 : '三',
        3 : '四',
        4 : '五',
        5 : '六',
        6 : '日'}
    try:
        date = datetime.strptime(time_str[:7] + '/' + str(day),"%Y/%m/%d").weekday()
    except:
        date = datetime.strptime(time_str[:7] + '/' + str(day-7),"%Y/%m/%d").weekday()
    cell_week = week_day_dict[date]
    return cell_day + ' ' + cell_week

def get_day_time(date_line, day):
    # 返回08:09-17:52、08:09-、-17:52
#     week_day = datetime.strptime(time_str[:7] + '/' + str(day),"%Y/%m/%d").weekday()
#     print(week_day)
    work_time = '-'
    # 多余的天数置空''，如2月份只有29天，则30号及31号置空
    if day>day_max:
        work_time = ''
        return work_time
    
    # 周六日默认置空''，若后续有时间则补上
    week_day = datetime.strptime(time_str[:7] + '/' + str(day),"%Y/%m/%d").weekday()
    if week_day==5 or week_day==6:
        work_time = ''
    
    if day>9:
        day = str(day)
    else:
        day = '0' + str(day)
    
    try:
        date_info = date_line[date_line['时间']==time_str[:7] + '/' + day]
#         print(date_info)
        work_begin = str(date_info['上班打卡时间'].values[0])
        work_end = str(date_info['下班打卡时间'].values[0])
        if work_begin == 'nan':
            work_time = '-' + work_end
        elif work_end == 'nan':
            work_time = work_begin + '-'
        else:
            work_time = work_begin + '-' + work_end
        print(work_time)
    except:
        pass
    return work_time

for i in dn_index:
#     print(i)
#     print(dn_index[i])
    # 部门
#     dp = dn_index[i][0]
    dp = i[0]
    try:
        # 设置sheet名
        worksheet = workbook.add_worksheet(dp)
        # 设置列宽度
        worksheet.set_column("A:G", 12.01)
        p_cell = 2
        w_cell = 3
    except:
        pass
    # 姓名
    name = i[1]
    # 打卡信息[时间 部门 姓名 上班打卡时间 下班打卡时间]
    nandf = df.groupby(['部门', '姓名']).get_group(i)
    # 剔除上下班都没时间的数据
    date_line = nandf[~nandf.isin(['--'])].dropna(thresh=4)
    print(date_line)
    
    #################### 填入表格 ####################
    # 字体大小format
    font_format = workbook.add_format({'font_size':9})
    # 字体居中format
    center_format = workbook.add_format({'align':'center'})
    # 下边框format
    bottom_format = workbook.add_format({'bottom':1})
    # 左边框format
    left_format = workbook.add_format({'left':1})
    # 右边框format
    right_format = workbook.add_format({'right':1})
    # 全边框format
    border_format = workbook.add_format({'align':'center', 'border':1, 'font_size':9})
    
    # 部门名称
    worksheet.write('A1', dp, font_format)
    # 合并居左A1-B1
#     merge_format = workbook.add_format({
#         'align':'left',#水平居左
#         'border':1
#     })
#     worksheet.merge_range('A1{}:B1{}', dp, merge_format)
    
    # 报表月份
    worksheet.write('F1', tb_date, font_format)
    
    # 部门员工名称
#     worksheet.write('D' + str(p_cell), name, center_format)
    worksheet.merge_range('A'+str(p_cell)+':G'+str(p_cell), name, border_format)
    
    day = 1
    # 一次填一行，重复4次
    for r in range(0,4):
        # 1号至28号星期格
        worksheet.write('A' + str(w_cell), get_day_and_week(day), border_format)
        worksheet.write('B' + str(w_cell), get_day_and_week(day+1), border_format)
        worksheet.write('C' + str(w_cell), get_day_and_week(day+2), border_format)
        worksheet.write('D' + str(w_cell), get_day_and_week(day+3), border_format)
        worksheet.write('E' + str(w_cell), get_day_and_week(day+4), border_format)
        worksheet.write('F' + str(w_cell), get_day_and_week(day+5), border_format)
        worksheet.write('G' + str(w_cell), get_day_and_week(day+6), border_format)
        # 1号至28号时间格
        worksheet.write('A' + str(w_cell+1), get_day_time(date_line, day), border_format)
        worksheet.write('B' + str(w_cell+1), get_day_time(date_line, day+1), border_format)
        worksheet.write('C' + str(w_cell+1), get_day_time(date_line, day+2), border_format)
        worksheet.write('D' + str(w_cell+1), get_day_time(date_line, day+3), border_format)
        worksheet.write('E' + str(w_cell+1), get_day_time(date_line, day+4), border_format)
        worksheet.write('F' + str(w_cell+1), get_day_time(date_line, day+5), border_format)
        worksheet.write('G' + str(w_cell+1), get_day_time(date_line, day+6), border_format)
        w_cell = w_cell+2
        day = day+7
    
    # 29号至31号星期格
    worksheet.write('A' + str(w_cell), get_day_and_week(29), border_format)
    worksheet.write('B' + str(w_cell), get_day_and_week(30), border_format)
    worksheet.write('C' + str(w_cell), get_day_and_week(31), border_format)
    # 29号至31号时间格
    worksheet.write('A' + str(w_cell+1), get_day_time(date_line, 29), border_format)
    worksheet.write('B' + str(w_cell+1), get_day_time(date_line, 30), border_format)
    worksheet.write('C' + str(w_cell+1), get_day_time(date_line, 31), border_format)
    
    p_cell = p_cell+11
    w_cell = w_cell+3
    
workbook.close()
    

              时间     部门  姓名 上班打卡时间 下班打卡时间
2133  2020/03/02  业务拓展部  喻宾  08:11  17:32
2134  2020/03/03  业务拓展部  喻宾  08:03  17:57
2135  2020/03/04  业务拓展部  喻宾  08:51  17:42
2136  2020/03/05  业务拓展部  喻宾  08:53  18:32
2137  2020/03/06  业务拓展部  喻宾  09:14  17:52
2138  2020/03/09  业务拓展部  喻宾  08:05  17:47
2139  2020/03/10  业务拓展部  喻宾  09:04  17:52
2140  2020/03/11  业务拓展部  喻宾  09:35  18:11
2141  2020/03/12  业务拓展部  喻宾  07:53  17:56
2142  2020/03/13  业务拓展部  喻宾  08:02  18:06
2143  2020/03/16  业务拓展部  喻宾  08:24  17:51
2144  2020/03/17  业务拓展部  喻宾  08:09  17:51
2145  2020/03/18  业务拓展部  喻宾  08:27  17:54
2146  2020/03/19  业务拓展部  喻宾  09:43  18:01
2147  2020/03/20  业务拓展部  喻宾  08:06    NaN
2148  2020/03/23  业务拓展部  喻宾  08:26  17:51
2149  2020/03/24  业务拓展部  喻宾  08:23  17:55
2150  2020/03/25  业务拓展部  喻宾  08:24  17:53
2151  2020/03/26  业务拓展部  喻宾  08:05  18:05
2152  2020/03/27  业务拓展部  喻宾  08:32  18:03
2153  2020/03/30  业务拓展部  喻宾  08:23  17:57
2154  2020/03/31  业务拓展部  喻宾  08:14  19:04
08:11-17:32
08:03-17:57
08:51-17:4